<a href="https://colab.research.google.com/github/MiguelAngeloTr/BIGDATA/blob/main/C1/Clase5AED_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark
!pip install -q findspark

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.window as W
import pyspark.sql.types as T
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
spark = SparkSession.builder.appName("AED PySpark").getOrCreate()
spark

In [ ]:
df = spark.read.csv('/content/titanic.csv', header=True, inferSchema=True)
print('Shape of the dataset: ', (df.count(), len(df.columns)))
df.show(n=10)

In [ ]:
df.columns

In [ ]:
#Viewing the dataframe schema
df.printSchema()

In [ ]:
df.describe().show()

# Manejo de datos faltantes

In [ ]:
def faltantes(df):
  falta = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns])
  return falta.show()

In [ ]:
faltantes(df)

In [ ]:
df_f1 = df.drop("Cabin","PassengerId",'Name')
faltantes(df_f1)
print('Shape of the dataset: ', (df_f1.count(), len(df_f1.columns)))

In [ ]:
df_f2 = df_f1.na.drop()
#df_f2 = df_f1.na.fill(0)
faltantes(df_f2)
print('Shape of the dataset: ', (df_f2.count(), len(df_f2.columns)))

# Algunas Consultas

## 1. Cuantas personas por clase y lugar de abordaje ingresaron al barco?

In [ ]:
res = df_f2.groupby("PClass","Embarked").count().orderBy("Pclass", "Embarked")
window = W.Window.partitionBy("Pclass").orderBy("Embarked")
result = res.withColumn("row", F.row_number().over(window)) \
               .withColumn("Pclass", F.when(F.col("row") == 1, F.col("Pclass")).otherwise(F.lit(""))) \
#               .drop("row")

# Mostrar el resultado
res.show()
#result.show(truncate=False)

## 2. Cuál es la edad promedio por sexo en cada clase en el barco?

In [ ]:
res = df_f2.groupby('Pclass', 'Sex').agg(F.mean('Age').alias('mean_Age')).orderBy('Pclass', 'Sex')
window = W.Window.partitionBy("Pclass").orderBy("Sex")
result = res.withColumn("row", F.row_number().over(window)) \
               .withColumn("Pclass", F.when(F.col("row") == 1, F.col("Pclass")).otherwise(F.lit(""))) \
               .drop("row")
result.show(truncate=False)

## 3. Cuál es el costo promedio del tiquete por sexo en cada clase en el barco?

In [ ]:
res = df_f2.groupby('Pclass', 'Sex').agg(F.mean('Fare').alias('mean_Fare')).orderBy('Pclass', 'Sex')
window = W.Window.partitionBy("Pclass").orderBy("Sex")
result = res.withColumn("row", F.row_number().over(window)) \
               .withColumn("Pclass", F.when(F.col("row") == 1, F.col("Pclass")).otherwise(F.lit(""))) \
               .drop("row")
result.show(truncate=False)

# Recodificación de Variables

## Creación de una variable Age_class, la cual a partir de la columna edad existente se obtenga una variable a partir de rangos de edad

In [ ]:
def age_condition(x):
    if x<18:
        return "18-"
    elif 18 <= x <= 25:
        return "18-25"
    elif 25 <= x <= 34:
        return "25-34"
    elif 35 <= x <= 44:
        return "35-44"
    elif 45 <= x <= 54:
        return "45-54"
    else:
        return "55+"

In [ ]:
age_udf = F.udf(lambda x: age_condition(x), T.StringType())
df_f3 = df_f2.withColumn("age_class", age_udf("Age"))
df_f3.show(5)

## 4. Realice de forma detallada por rango de edad, cuantas personas ingresaron en el barco y que porcentaje representan. Ademas, cual fue el costo máximo, mínimo y promedio de los tiquetes en cada caso

In [ ]:
window = W.Window.rowsBetween(W.Window.unboundedPreceding,W.Window.unboundedFollowing)
tab = df_f3.select(['age_class','Fare']).\
      groupBy('age_class').\
      agg(F.count('Fare').alias('Quantity'),
      F.mean('Fare').alias('Fare_avg'),
      F.min('Fare').alias('Fare_min'),
      F.max('Fare').alias('Fare_max')).\
      withColumn('Total',F.sum(F.col('Quantity')).over(window)).\
      withColumn('Percent',F.col('Quantity')*100/F.col('Total')).\
      drop(F.col('Total')).orderBy("age_class")
tab.show()

## ¿Como la tabla anterior podría ser desagregada por clase (primera, segunda y tercera clase)?

# Gráficos: Cajas e Histogramas

In [ ]:
df_Age = df_f2.select('Age').toPandas()
df.select(F.skewness('Age'),F.kurtosis('Age')).show()
plt.figure(figsize=(7, 4))
plt.subplot(1, 2, 1)
sns.histplot(df_Age, kde=True, bins=10, color='blue')
plt.title('Histograma')

# Diagrama de caja (boxplot)
plt.subplot(1, 2, 2)
sns.boxplot(df_Age, color='orange')
plt.title('Diagrama de Caja')

# Mostrar los gráficos
plt.tight_layout()
plt.show()

In [ ]:
df_Fare = np.log1p(df_f2.select('Fare').toPandas())
df.select(F.skewness('Fare'),F.kurtosis('Fare')).show()
plt.figure(figsize=(7, 4))
plt.subplot(1, 2, 1)
sns.histplot(df_Fare, kde=True, bins=10, color='blue')
plt.title('Histograma')

plt.subplot(1, 2, 2)
sns.boxplot(df_Fare, color='orange')
plt.title('Diagrama de Caja')

plt.tight_layout()
plt.show()

## Cual fue el costo del tiquete en relacion a la clase en el barco?

In [ ]:
df_class_fare = df_f2.select('Pclass','Fare').toPandas()
sns.boxplot(x='Pclass', y='Fare', data=df_class_fare, hue='Pclass')

## Cual fue la proporción de hombres y mujeres en el barco?

In [ ]:
df_Sex = df_f3.select('Sex','age_class').toPandas()
plt.figure(figsize=(7, 5))
sns.countplot(x='Sex', data=df_Sex, hue='age_class')
plt.show()

## Existe alguna relación entre la edad y el costo del tiquete para el pasajero?

In [ ]:
df_fare = df_f2.select('Age','Fare','Sex').toPandas()
plt.figure(figsize=(7, 5))
sns.scatterplot(x='Age', y='Fare', data=df_fare, hue='Sex')
plt.show()

## Que variables tuvieron relación con la superviviencia de las personas en el barco?

In [ ]:
df_full = df_f2.select('Age','Fare','SibSp','Parch','PClass','Survived').toPandas()
sns.heatmap(df_full.corr(), annot=True)
plt.show()